
# Assignment 2 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [1]:
import numpy as np

matrix = np.array([[0.3, 0.4, 0.3], [0.2, 0.5, 0.3], [0.4, 0.3, 0.3]])
matrix

array([[0.3, 0.4, 0.3],
       [0.2, 0.5, 0.3],
       [0.4, 0.3, 0.3]])

In [2]:
initial_state = np.array([0, 1, 0])
initial_state

array([0, 1, 0])

In [3]:
transition = initial_state @ np.linalg.matrix_power(matrix, 2)
transition

array([0.28, 0.42, 0.3 ])

In [4]:
# Part 1

# Fill in the answer to part 1 below as a decimal number
problem1_p1 = transition[0]
problem1_p1

np.float64(0.28)

In [5]:
# Path 1: Suburbs -> Countryside -> Downtown
# Path 2: Suburbs -> Suburbs -> Downtown

path1_1 = (initial_state @ np.linalg.matrix_power(matrix, 1))[2]
path2_1 = (initial_state @ np.linalg.matrix_power(matrix, 1))[1]

In [6]:
path1_2 = (np.array([0, 0, 1]) @ np.linalg.matrix_power(matrix, 1))[0]
path2_2 = (np.array([0, 1, 0]) @ np.linalg.matrix_power(matrix, 1))[0]

In [7]:
# Part 2

# Fill in the answer to part 2 below as a decimal number
problem1_p2 = path1_1 * path1_2 + path2_1 * path2_2
problem1_p2

np.float64(0.22)

In [8]:
check = np.all(np.linalg.matrix_power(matrix, len(matrix[0])) > 0)
check

np.True_

In [9]:
# Part 3

# Fill in the answer to part 3 below as a boolean
problem1_irreducible = check
problem1_irreducible

np.True_

In [10]:
eval, evec = np.linalg.eig(matrix.T)
print(eval, evec)

[ 1.00000000e+00 -9.90532031e-17  1.00000000e-01] [[-4.93651112e-01 -8.16496581e-01  7.07106781e-01]
 [-7.02503505e-01  4.08248290e-01 -7.07106781e-01]
 [-5.12637693e-01  4.08248290e-01  4.40033516e-16]]


In [11]:
stationary = evec[:, np.isclose(eval, 1)]
stationary

array([[-0.49365111],
       [-0.70250351],
       [-0.51263769]])

In [12]:
stationary = stationary[:, 0].real
stationary

array([-0.49365111, -0.70250351, -0.51263769])

In [13]:
stationary = stationary / np.sum(stationary)
stationary

array([0.28888889, 0.41111111, 0.3       ])

In [14]:
# Part 4

# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
problem1_stationary = stationary
problem1_stationary

array([0.28888889, 0.41111111, 0.3       ])

In [15]:
current_state = [0, 1, 0]
expected_steps = 0
for i in range(1, 31):
    current_state = current_state @ matrix
    expected_steps += i * current_state[0]
    current_state[0] = 0

expected_steps

np.float64(3.8444520365145998)

In [16]:
# Part 5

# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
problem1_ET = expected_steps
problem1_ET

np.float64(3.8444520365145998)

---
## Assignment 2, PROBLEM 2
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example (in `07-Optimization.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  

In [17]:
import numpy as np
from scipy import optimize

# do NOT change the next three lines
np.random.seed(123456)  # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10, 2, 30)


# define the negative log likelihoo function you want to minimise by editing XXX
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    mu_param = parameters[0]
    sigma_param = parameters[1]
    sum = 0
    for i in range(len(normallySimulatedDataSamples)):
        term = (1 / (np.sqrt(2 * np.pi * sigma_param))) * np.e ** (
                (-1 / (2 * sigma_param)) * (normallySimulatedDataSamples[i] - mu_param) ** 2)
        sum += np.log(term)

    return -sum


# you should only change XXX below and not anything else
parameter_bounding_box = ((-100, 100), (0.01, 100))  # specify the constraints for each parameter - some guess work...
initial_arguments = np.array([1, 1])  # point in 2D to initialise the minimize algorithm
result_problem2_opt = optimize.minimize(negLogLklOfIIDNormalSamples, initial_arguments, bounds=parameter_bounding_box)
# call the minimize method above finally! you need to play a bit to get initial conditions and bounding box ok
result_problem2_opt


/var/folders/zz/36b01_n10d5_rkg9ld_51cj80000gn/T/ipykernel_38697/2267402195.py:19: RuntimeWarning: divide by zero encountered in log
  sum += np.log(term)
/Users/madhurgupta/my_env/lib/python3.12/site-packages/scipy/optimize/_numdiff.py:592: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 74.52943250814468
        x: [ 6.923e+00  8.517e+00]
      nit: 11
      jac: [-8.263e+00  1.990e-02]
     nfev: 51
     njev: 17
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment2Problem3(x)`, where `x` is a list of $n$ input data points.

In [18]:
# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''write comment of what this function does'''

    return (5 * len(x)) / sum(x)

In [19]:
MLeForAssignment2Problem3([2.0, 3.0, 4.0, 5.0])

1.4285714285714286

---
## Assignment 2, PROBLEM 4
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [20]:

def problem4_LCG(size=None, seed=0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """

    a, b, M = 1664525, 1013904223, 2 ** 31
    numbers = []
    X = seed
    for _ in range(size):
        X = (a * X + b) % M
        numbers.append(X)

    return numbers

In [21]:
problem4_LCG(100, 42)

[1083814273,
 378494188,
 331920219,
 955863294,
 1613448261,
 110225632,
 1921058495,
 508781842,
 1605517641,
 2124438036,
 1516994147,
 2146095206,
 609889421,
 1552442504,
 414334535,
 389768954,
 602196497,
 1933114428,
 1823451755,
 1672756430,
 216931797,
 775312688,
 215115471,
 1112253922,
 1903193049,
 1380836196,
 666982259,
 1414739510,
 2005415965,
 260725464,
 96444503,
 160154058,
 1417168545,
 1483190156,
 777037179,
 1635343518,
 367893349,
 843519360,
 907807455,
 938957490,
 2105768553,
 1931673780,
 420590723,
 822890502,
 1921476525,
 545465640,
 1154336359,
 402526362,
 1308436273,
 1672509148,
 926721163,
 1168486510,
 81968373,
 1193880016,
 1434383087,
 1644397442,
 1957981433,
 827634180,
 1594725779,
 353574358,
 1446034237,
 1350026104,
 555593847,
 2061456234,
 442264513,
 1810387500,
 1313122203,
 1859532862,
 902201989,
 1317114400,
 509353727,
 39657554,
 504120201,
 1045952340,
 447085219,
 849649574,
 2107454157,
 900543944,
 976248455,
 834439738,
 16

In [22]:

def problem4_uniform(generator=None, period=1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """

    samples = generator(size, seed)
    uniform_samples = [sample / (period - 1) for sample in samples]
    return uniform_samples

In [23]:

def problem4_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution.
    Runs n_iterations
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    n_iterations : an integer denoting how many attempts should be made in the accept-reject sampler
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """

    samples = uniformGenerator(n_iterations, seed)

    P_x = []
    for i in range(n_iterations):
        x = samples[i]
        p_x = (np.pi / 2) * np.abs(np.sin(2 * np.pi * x))

        u = uniformGenerator(n_iterations, seed)[0]
        if u <= 10 * (x / p_x):
            P_x.append(p_x)
    return P_x

---
#### Local Test for Assignment 2, PROBLEM 4
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [24]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem4_LCG(size=10, seed=1))

period = 2 ** 31

print("Uniform sampler %s" % problem4_uniform(generator=problem4_LCG, period=period, size=10, seed=1))

uniform_sampler = lambda size, seed: problem4_uniform(generator=problem4_LCG, period=period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=uniform_sampler, n_iterations=20, seed=1))

LCG output: [1015568748, 1586005467, 18219390, 879966917, 217083232, 1587069247, 1180097938, 241328073, 70837908, 598057187]
Uniform sampler [0.4729110507634054, 0.7385413477842423, 0.008484064605312452, 0.40976652754925497, 0.10108725731311705, 0.7390367089486852, 0.5495259252141816, 0.11237714118900576, 0.03298647144482773, 0.27849207971174833]
Accept-Reject sampler [np.float64(0.2660682045879033), np.float64(1.5667269389103675), np.float64(0.08369470997672833), np.float64(0.8436194236014133), np.float64(0.9319507026845661), np.float64(1.5670710395147722), np.float64(0.4809506854291521), np.float64(1.0192282610638725), np.float64(0.3232375669339179), np.float64(1.545692626881825), np.float64(0.00890513092142037), np.float64(1.3005297644147658), np.float64(1.427046275454105), np.float64(1.3944791611282086), np.float64(0.2928656996016643), np.float64(0.3744453866368082), np.float64(1.546158416110423), np.float64(1.2922063501622314), np.float64(1.1769298242950408), np.float64(0.60492916

In [25]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size, seed):
    import random
    random.seed(seed)

    return [random.uniform(0, 1) for s in range(size)]


print(
    "Accept-Reject sampler %s" % problem4_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [np.float64(1.1741128167850454), np.float64(1.2855225073129173), np.float64(1.5649168633439452), np.float64(1.5699996871810427), np.float64(0.04504770738210951), np.float64(0.4901774164088657), np.float64(1.279978281538078), np.float64(1.5245313883405471), np.float64(0.8735867736904243), np.float64(0.27830144010573704), np.float64(1.3481908954994044), np.float64(0.6440020074783206), np.float64(1.5661228863357384), np.float64(0.020785306808902768), np.float64(0.5284911789780555), np.float64(1.545749030558017), np.float64(1.5568318950366071), np.float64(0.5295738007987892), np.float64(0.9118561433959141), np.float64(0.3000556048121108)]
